Тест

In [1]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

from sloyka import EmotionRecognizer

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kozak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2024-06-03 13:18:04,299 SequenceTagger predicts: Dictionary with 7 tags: O, S-Service, B-Service, E-Service, I-Service, <START>, <STOP>


In [2]:
#загрузка датафрейма
import pandas as pd
df = pd.read_csv('C:/Users/kozak/OneDrive/Рабочий стол/итмо поступление/НИРМА/Комменты и посты/Парсер/Индентичность/comments_grazhdanka_spb_kalininsky.csv', sep = ';', index_col=0)

In [6]:
df = df.head(100)

In [7]:
df

,name,last_name,date,text,post_id
1370336,Sergey,Nazarov,2023-01-17,Русская Гражданка заканчивалась сразу за кварт...,1370265.0
1370324,Anna,Fomina,2023-01-17,Маленькая школа получилась.. всего 550 мест..,1370271.0
1370319,Viktoria,Agu,2023-01-17,"Главное, что бы все вышеперечисленное работало...",1370271.0
1370278,Irina,Vladimirovna,2023-01-16,"А кто знает,почему от этой школы шарахались,ка...",1370271.0
1370412,Alya,Dzhafarova,2023-01-17,"Интересно ,а как в нее можно попасть?Если в 3 ...",1370271.0
...,...,...,...,...,...
1370825,Nina,Khryakova,2023-01-19,Очень вкусная и отличная продукция все натурал...,1370426.0
1370547,Vika,Lee,2023-01-18,Не жрать что попало ))),1370468.0
1370537,Vyacheslav,Slivkin,2023-01-17,Есть в основной массе белки и сложные углеводы...,1370468.0
1370529,Vitek,Sokol,2023-01-17,Первитин или аналоги.,1370468.0


Быстрый способо определить эмоцию, используя одну модель. По умолчанию стоит модель HuggingFaceModel.Text.Bert_Large, но можно использовать также HuggingFaceModel.Text.Bert_Tiny, HuggingFaceModel.Text.Bert_Base, HuggingFaceModel.Text.Bert_Tiny2

In [9]:
recognizer = EmotionRecognizer()
df['emotion'] = df['text'].apply(recognizer.recognize_emotion)

In [10]:
df

,name,last_name,date,text,post_id,emotion
1370336,Sergey,Nazarov,2023-01-17,Русская Гражданка заканчивалась сразу за кварт...,1370265.0,neutral
1370324,Anna,Fomina,2023-01-17,Маленькая школа получилась.. всего 550 мест..,1370271.0,sadness
1370319,Viktoria,Agu,2023-01-17,"Главное, что бы все вышеперечисленное работало...",1370271.0,neutral
1370278,Irina,Vladimirovna,2023-01-16,"А кто знает,почему от этой школы шарахались,ка...",1370271.0,enthusiasm
1370412,Alya,Dzhafarova,2023-01-17,"Интересно ,а как в нее можно попасть?Если в 3 ...",1370271.0,enthusiasm
...,...,...,...,...,...,...
1370825,Nina,Khryakova,2023-01-19,Очень вкусная и отличная продукция все натурал...,1370426.0,happiness
1370547,Vika,Lee,2023-01-18,Не жрать что попало ))),1370468.0,happiness
1370537,Vyacheslav,Slivkin,2023-01-17,Есть в основной массе белки и сложные углеводы...,1370468.0,neutral
1370529,Vitek,Sokol,2023-01-17,Первитин или аналоги.,1370468.0,neutral


Способ долгий, но возможно более точный. Берёт среднее из нескольких моделей и возвращает доминирующую эмоцию.

In [3]:
#Используются все 4 модели и считается среднее
df = df.head(4)
recognizer = EmotionRecognizer()
df['emotion'] = df['text'].apply(lambda text: recognizer.recognize_average_emotion_from_multiple_models(text))

In [4]:
df

,name,last_name,date,text,post_id,emotion
1370336,Sergey,Nazarov,2023-01-17,Русская Гражданка заканчивалась сразу за кварт...,1370265.0,neutral
1370324,Anna,Fomina,2023-01-17,Маленькая школа получилась.. всего 550 мест..,1370271.0,neutral
1370319,Viktoria,Agu,2023-01-17,"Главное, что бы все вышеперечисленное работало...",1370271.0,neutral
1370278,Irina,Vladimirovna,2023-01-16,"А кто знает,почему от этой школы шарахались,ка...",1370271.0,enthusiasm


In [6]:
#Используются 1-4 модели, задаётся ввод вручную
from aniemore.models import HuggingFaceModel
models = [
    HuggingFaceModel.Text.Bert_Tiny,
    HuggingFaceModel.Text.Bert_Base
]
recognizer = EmotionRecognizer()

df = df.head(3)
df['emotion'] = df['text'].apply(lambda text: recognizer.recognize_average_emotion_from_multiple_models(text, models))

In [7]:
df

,name,last_name,date,text,post_id,emotion
1370336,Sergey,Nazarov,2023-01-17,Русская Гражданка заканчивалась сразу за кварт...,1370265.0,neutral
1370324,Anna,Fomina,2023-01-17,Маленькая школа получилась.. всего 550 мест..,1370271.0,neutral
1370319,Viktoria,Agu,2023-01-17,"Главное, что бы все вышеперечисленное работало...",1370271.0,neutral
